# Local search raport

Łukasz Andryszewski 151930

## Qaudratic Assignment Problem (QAP)

The tackled problem is NP-hard, meaning there's no polynomial time algorithm that guarantees optimum. The goal of the optimization is:

$$ \min_{\pi} \sum_{i=0}^{n}\sum_{j=0}^{n} a_{ij} \cdot b_{\pi_{i}\pi_{j}}$$

The problem can be thought of as assigning some 'departments' to some 'locations'. The $a$ is the flow of communication between the 'departments' and $b$ is a distance matrix between the 'locations'. The $\pi$ is a permutation, which describes to which 'locations', different 'departments' are assigned. The aim is to minimise the cost of communication between all of the 'departments', by controlling their 'location'.

The problem has real world applications, like in the case of the $\texttt{KraXXx}$ instances. These contain real world data, which was used to plan the Klinikum Regensburg in Germany. Or the $\texttt{EscXXx}$ instances which "...stem from an application in computer science, from the testing of self-testable sequential
circuits".

### Chosen instances:

Instances bigger in size were selected, to limit test the solver. For each type, as described in the QAPLIB description, it was desirable to have it in multiple sizes in order to potentially analyze the impact of the size of instance. It was also important for the instance to have the optimum known.

- $\texttt{Lipa20a}$, $\texttt{Lipa50a}$ and $\texttt{Lipa90a}$ examples were chosen because they are generated using the same method, the optimum is known and they are assymetric which makes them somewhat interesting.

- $\texttt{Esc16a}$, $\texttt{Esc32g}$ and $\texttt{Esc128}$, because they come from real life problems and are of different sizes.

- $\texttt{tai50b}$, $\texttt{tai100b}$ and $\texttt{tai150b}$, because they are all assymetric and generated the same way

- $\texttt{tai64c}$, $\texttt{tai256c}$ because they 'occur in the generation of grey patterns' and also contain the largest instance

## Solver implementation

The solver used to analyse the problem was written in C++ and compiled using g++. It can appropriately handle assymetric instances. It implements:

- Local search (in steepest and greedy version) - local_search
- Random walk - random_walk
- Random search - random_seach
- a construction Heuristic - heuristic

Construction heuristic calculates value of adding a new assignment at the end. Then the best assignment is kept. An additional boolean array is used to determine if a 'location' is already assigned. Additionally the first assignment is random to make the heuristic non-deterministic.

The program is used as follows:

```bat
bio_solver.exe (instance:str) (solver_name:std) (repetitions:int) (solver_args...)
```

Local search takes ```steepest``` or ```greedy``` as an argument. Random walk and random search take ```duration``` in nanoseconds as an argument.

The program outputs:

- instance_name instance_size optimal_value
- optimal_solution
- number_of_repetitions
Then repetitions of:
- initial_solution
- final_solution
- starting_value final_value
- evaluations steps execution_time

```run_all.bat``` program takes instances as an argument and then runs all the solvers on these instances, with predefined number of repetitions. The execution time for RS and RW is taken as an average of execution times of greedy and steepest local search.

# Neighbourhood

The defining operator in local search is how the neighbourhood of a solution is defined. Local search implemented hear used the 2-OPT neighbourhood. For this operator the size can be defined as:

$$ N = \dfrac{n^2 - n}{2} $$

Which can be thought of as the upper triangular part of a square matrix. The neighbourhood is initialized, for each instance of local search, by creating every combination of two positions. Then its shuffled. A random offset is initialized to help with the randomization of ordering, whilst not spending much time on additional reshuffling. This is especially important for greedy, because the first found improvement is selected in each iteration.

# Comparison of algorithms

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from util import pandify
from itertools import product

In [ ]:
def quality_over(start_val, final_val, opt):
    # assuming minimization of objective
    # to be minimized
    return (start_val - final_val) / opt

def quality_to_opt(final_val, opt):
    return quality_over(final_val,opt,opt)

def similarity(sol1, sol2):
    return np.mean(sol1==sol2)

In [ ]:
instances = ["lipa20a","lipa50a","lipa90a","esc16a","esc32g","esc128","tai50b","tai100b", "tai150b","tai64c","tai256c"]
print(*instances)

In [ ]:
#instances = list("esc128 tai256c wil100 tho150 lipa90b chr25a bur26a rou12".split())
solvers = ["ls_greedy", "ls_steepest", "rs", "rw", "heuristic"]
shorter_solvers = ["G","S","RS","RW", "H"]
full_solvers = ["local_search_greedy","local_search_steepest","random_search","random_walk", "heuristic"]
solver_map = {short:full for short,full in zip(solvers,full_solvers)}
solver_to_short_map = {full:short for short,full in zip(shorter_solvers,full_solvers)}
colors = ["seagreen", "mediumblue", "red", "orange", "hotpink"]
color_map = {solver:color for solver,color in zip(colors, full_solvers)}

In [ ]:
data = pandify(instances,solvers,solver_map)
data.head(100)

## Running times

The time was measured in nanoseconds with the C++ ```chrono``` library using a ```high_resolution_clock```.

In [ ]:
flierprops = dict(marker='o', markerfacecolor='black', markersize=5,markeredgecolor='none',alpha=0.2)
X = data.pivot(columns=["instance","solver"], index="repetition")["time"]
fig, axs = plt.subplots(ncols=3,nrows=4,layout="tight",sharex=True,sharey='row')
fig.set_size_inches(10,10)
fig.supylabel("Running time")
for instance, ax in zip(instances,fig.get_axes()):
    ax.set_title(instance)
    vplot = ax.violinplot(X[instance],showextrema=False)
    bplot = ax.boxplot(X[instance],widths=0.2,medianprops=dict(color="black"),flierprops=flierprops)
    ax.set_yscale('log')
    ax.grid(True,which="major",ls='-')
    ax.grid(True,which="minor",ls='dotted')
    ax.yaxis.set_major_locator(mpl.ticker.LogLocator())
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda x, pos: f"{x/1e6:.2f} ms"))
    ax.set_xticks(np.arange(len(solvers))+1,shorter_solvers[:len(solvers)])
    for vp, color in zip(vplot["bodies"],colors):
        vp.set_color(color)
    ax.set_facecolor("whitesmoke")
fig.delaxes(axs[-1, -1])
axs[-2,-1].xaxis.set_tick_params(labelbottom=True)
plt.show()

As it turns out steepest is slower than greedy on every tested instance. The running times of random searche and random walk are identical in each repetition. The chosen duration of the algorithms was the average between mean running times of both local searches. As expected the construction heuristic was by far the fastest.

## Quality

The quality is measured as a relative gap according to the following formula:

$$ Q(v_z) = \dfrac{v_z-v_o}{v_o} = \dfrac{v_z}{v_o} - 1$$

where $v_z$ is the solution value and $v_o$ are the value of the optimum. In describes the distance to the optimum relative to it. This way it is also better comparable between instances. Lower values are more desirable.

In [ ]:
X = data.assign(quality=lambda df: quality_to_opt(df.final_value,df.optimal_value)).pivot(columns=["instance","solver"], index="repetition")["quality"]
X = X.fillna(X.max())
fig, axs = plt.subplots(nrows=4,ncols=3,layout="tight",sharex=True)
fig.set_size_inches(10,10)
fig.supylabel("Quality")
for instance, ax in zip(instances,fig.get_axes()):
    ax.set_title(instance)
    vplot = ax.violinplot(X[instance],showextrema=False)
    bplot = ax.boxplot(X[instance],widths=0.2,medianprops=dict(color="black"),flierprops=flierprops)
    ax.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax=1))
    ax.set_xticks(np.arange(len(solvers))+1,shorter_solvers[:len(solvers)])
    for vp, color in zip(vplot["bodies"],colors):
        vp.set_color(color)
    ax.set_facecolor("whitesmoke")
    ax.grid()
fig.delaxes(axs[-1, -1])
axs[-2,-1].xaxis.set_tick_params(labelbottom=True)
plt.show()

As expected the random walk and random search usually have the biggest variances when it comes to the values of the solutions. However in some cases the heuristic performed worse than them. Both local searches had very similar performances and similar distributions, although greedy seems to be slightly better. For a few instances, like $\texttt{esc32g}$ the quality of the heuristic reaches $0\%$, which means it achieves the optimal value. This could suggest that these instances may be trivial.

## Efficiency of algorithms

The quality over time is measured as difference of relative gaps between the initial and final solution int relation to the optimum:

$$ V(v_i,v_z,t) = \dfrac{Q(v_i)-Q(v_z)}{t} = \dfrac{(v_i-v_o)-(v_z-v_o)}{v_ot} = \dfrac{v_i-v_z}{v_ot}$$

where $v_z$ is the solution value and $v_o$ are the value of the optimum. Intuitively it can be though of as 'speed' of the algorithm - how much it improves the initial solution over some time, relative to optima. Higher values are more desirable here.

In [ ]:
X_raw = data.assign(quality=lambda df: quality_over(df.initial_value,df.final_value,df.optimal_value)).assign(q_over_t=lambda df: df.quality/df.time)
X = X_raw.pivot(columns=["instance","solver"], index="repetition")["q_over_t"]
fig, axs = plt.subplots(nrows=4,ncols=3,layout="tight",sharex=True,sharey='row')
fig.set_size_inches(10,10)
fig.supylabel("Quality over time")
for instance, ax in zip(instances,fig.get_axes()):
    ax.set_title(instance)
    vplot = ax.violinplot(X[instance],showextrema=False)
    bplot = ax.boxplot(X[instance],widths=0.2,medianprops=dict(color="black"),flierprops=flierprops)
    ax.set_xticks(np.arange(len(solvers))+1,shorter_solvers[:len(solvers)])
    ax.set_yscale('log')
    ax.grid(True,which="major",ls='-')
    ax.grid(True,which="minor",ls='dotted')
    ax.yaxis.set_major_locator(mpl.ticker.LogLocator())
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda x, pos: f"{x*100:.0e} %/ns"))
    # ax.xaxis.set_major_locator(mpl.ticker.FixedLocator(np.arange(1,len(selected)+1)))
    # ax.set_xticklabels(shorter_solvers)
    for vp, color in zip(vplot["bodies"],colors):
        vp.set_color(color)
    ax.set_facecolor("whitesmoke")
    ax.grid()
fig.delaxes(axs[-1, -1])
axs[-2,-1].xaxis.set_tick_params(labelbottom=True)
plt.show()

Although greedy and steepest achieve similar qualities, because greedy is much faster, it achieves much better efficiency overall. Actually, in effciency steepest performs similarily to random search and random walk. In some cases the heuristic, when compared to a random initial solution, produces a worse solution. This cannot be exactly visualized on a logarithm scale, because it reaches negative values. It can be seen on the table below:

In [ ]:
X_raw[X_raw.solver == "heuristic"].pivot(columns="instance",index="repetition")["q_over_t"].describe()

## Number of iterations of Local Searches

Here the number of iterations performed by local searches are compared. Each iteration is defined as a single move to a neighbour. For steepest the entire neighbourhood is checked and the one giving the best improvement is performed. For greedy the first found improving move is performed.

In [ ]:
selected = full_solvers[:2]
mask_ls = data["solver"].isin(selected)
X = data[mask_ls].pivot(columns=["instance","solver"], index="repetition")["iterations"]
fig, axs = plt.subplots(nrows=4,ncols=3,layout="tight",sharex=True)
fig.set_size_inches(10,18)
fig.supylabel("Iterations")
for instance, ax in zip(instances,fig.get_axes()):
    ax.set_title(instance)
    vplot = ax.violinplot(X[instance],showextrema=False)
    bplot = ax.boxplot(X[instance],widths=0.2,medianprops=dict(color="black"))
    #ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda x, pos: f"{x*100:.3e} %/ns"))
    ax.set_xticks(np.arange(len(selected))+1,shorter_solvers[:len(selected)])
    for vp, color in zip(vplot["bodies"],colors[:2]):
        vp.set_color(color)
    ax.set_facecolor("whitesmoke")
    ax.grid()
fig.delaxes(axs[-1, -1])
axs[-2,-1].xaxis.set_tick_params(labelbottom=True)
plt.show()

As expected greedy peforms more iterations of local search, as it does not check the entire neighbourhood. Because of that it does not converge as quickly as steepest.

## Number of evaluations

An evaluation is counted each time an algorithm either evaluates an entire solution or if an algorithm performs a partial evaluation (calculating the delta of a swap).

In [ ]:
# selected = full_solvers[:-1]
# mask_GSRSRW = data["solver"].isin(selected)
X = data.pivot(columns=["instance","solver"], index="repetition")["evaluations"]
fig, axs = plt.subplots(nrows=4,ncols=3,layout="tight",sharex=True)
fig.set_size_inches(10,10)
fig.supylabel("Evaluations")
for instance, ax in zip(instances,fig.get_axes()):
    ax.set_title(instance)
    vplot = ax.violinplot(X[instance],showextrema=False)
    bplot = ax.boxplot(X[instance],widths=0.2,medianprops=dict(color="black"))
    #ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda x, pos: f"{x*100:.3e} %/ns"))
    ax.set_xticks(np.arange(len(shorter_solvers))+1,shorter_solvers)
    for vp, color in zip(vplot["bodies"],colors):
        vp.set_color(color)
    ax.set_facecolor("whitesmoke")
    ax.grid()
fig.delaxes(axs[-1, -1])
axs[-2,-1].xaxis.set_tick_params(labelbottom=True)
plt.show()

Despite steepest performing more iterations, it performs less evaluations than steepest. This is exactly because it does not check the entire neighbourhood. It is also expected for random walk to perform more iterations than random search. As these are run for the same amount of time and random walk simply peforms much less in a single 'iteration' compared to random search, it is able to evaluate more solutions. In some cases like $\texttt{Esc32g}$ the heuristic performs more evaluations than random search.

## Quality of initial vs final solution

The dependence between the initial quality of solution and final quality for local searches is checked.

In [ ]:
X = data[mask_ls].assign(
    quality_fin=lambda df: quality_to_opt(df.final_value,df.optimal_value)
    ).assign(
    quality_start=lambda df: quality_to_opt(df.initial_value,df.optimal_value)
    ).pivot(columns=["instance","solver"], index="repetition")
corr_data = {col:[] for col in ["instance","solver","correlation"]}
fig, axs = plt.subplots(nrows=4,ncols=3,layout="constrained")
fig.set_size_inches(10,10)
fig.supxlabel("Initial quality")
fig.supylabel("Final quality")
for instance, ax in zip(instances, fig.get_axes()):
    ax.set_title(instance)
    for solver, color in zip(full_solvers[:2], colors):
        x = X["quality_start"][instance][solver]
        y = X["quality_fin"][instance][solver]
        ax.scatter(x, y, s=3, label=solver_to_short_map[solver],color=color)
        corr = np.corrcoef(x,y)[0,1]
        corr_data["instance"].append(instance)
        corr_data["solver"].append(solver)
        corr_data["correlation"].append(corr)
        #ax.text(0.9,0.1,f"{corr:2f}",transform=ax.transAxes)
    ax.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax=1))
    ax.xaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax=1))
    #ax.tick_params(axis="x", labelrotation=15)
    ax.grid()
    ax.set_facecolor("whitesmoke")
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='lower right')
fig.delaxes(axs[-1, -1])
plt.show()
corr_data = pd.DataFrame.from_dict(corr_data)

Additionally correlation between the final and initial solution value is presented. The exact measure used is the Pearson Correlation coefficient.

In [ ]:
corr_data.pivot(columns="instance",index="solver")

It would be expected that better initial solutions would produce better final solutions, by the possibility of being placed in a better region of the solution space. However there is seemingly no significant correlation between the quality of initial solution and the quality of the final solution, at least for the selected instances.

## Multi-random start local search

In this section the 300 repetitions of local searches are treated as a single multi-random start local search. It works similarily to random search but each solution in the restart is a local optimum.

In [ ]:
X = data[mask_ls].assign(quality=lambda df: quality_to_opt(df.final_value,df.optimal_value)).pivot(columns=["instance","solver"], index="repetition")["quality"]
best = X.cummin()
mean = X.expanding().mean()
std = X.expanding().std().fillna(0)
fig, axs = plt.subplots(nrows=4,ncols=3,layout="constrained",sharex=True)
fig.set_size_inches(10,10)
fig.supxlabel("Repetitions")
fig.supylabel("Quality")
for instance, ax in zip(instances, fig.get_axes()):
    ax.set_title(instance)
    ax.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax=1))
    for solver, color in zip(full_solvers[:2], colors):
        u = mean[instance][solver]
        ro = std[instance][solver]
        ax.plot(best[instance][solver],label=solver_to_short_map[solver]+" best",color=color)
        ax.plot(u,'--',label=solver_to_short_map[solver]+" mean",color=color)
        ax.fill_between(np.arange(len(u)),u+ro,(u-ro).clip(0), label=solver_to_short_map[solver]+" std",alpha=0.1,color=color)
    ax.grid()
    ax.set_facecolor("whitesmoke")
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='lower right',ncols=2)
fig.delaxes(axs[-1, -1])
axs[-2,-1].xaxis.set_tick_params(labelbottom=True)
plt.show()

There is value to be gained from running the local search algorithm multiple times. However the exact number of steps is highly dependent on the instance, as it is an additional parameter. For most of the selected instances it seems $100$ repetitions is enough, as the algorithm usually flatlines. The mean stabilizes and stays roughly the same. Standard deviation is also stable and its value ultimately depends on the instance.

## Similarity of local optimas

The similarity between instances is measured as the inverse of Hamming distance:

$$ {Sim}(x_1, x_2) = 1 - H(x_1,x_2) = 1 - \dfrac{\sum_{i=0}^n \begin{cases} 1 & \text{if $x_1[i]$ $\neq$ $x_2[i]$} \\ 0 & {otherwise}\end{cases} }{n}$$

This measure seems appropriate for the nature of the problem as the position of a solution $x$ describes the department, while the index at that position describes to which location that department is assigned.

The similarities are calculated between all local optima and then averaged, as well as between the local optimum and global optimum.

The selected instances are $\texttt{Lipa90a}$ and $\texttt{tai100b}$ as the heuristic performs badly on them.

In [ ]:
subset = ["lipa20a","lipa90a","esc16a", "tai100b"]
selected = full_solvers[:2]
to = ["optimal_solution","final_solution"]
to_titles = ["Wrt optimum", "Wrt eachother"]
to_map = {wrt:title for wrt,title in zip(to,to_titles)}
X = data[data["repetition"]<100].assign(
    quality=lambda df: quality_over(df.initial_value,df.final_value,df.optimal_value)
    ).pivot(columns=["instance","solver"], index="repetition")[to+["quality"]]
fig, ax = plt.subplots(nrows=4,ncols=2,layout="constrained")
fig.set_size_inches(6,12)
for (instance, wrt), ax in zip(product(subset,to),fig.get_axes()):
    for solver, color in zip(selected,colors):
        y = np.array(list(map(lambda sols: similarity(*sols),product(X[wrt][instance][solver],X["final_solution"][instance][solver]))))
        n = np.ceil(np.sqrt(y.shape[0])).astype(int)
        y = y.reshape((n,n))
        np.fill_diagonal(y,0)
        mean_y = (y.mean(0))*(n/(n-1))
        x = X["quality"][instance][solver]
        ax.scatter(x,mean_y,label=solver_to_short_map[solver],color=color,s=4)
        if wrt == to[1]:
            np.fill_diagonal(y,mean_y)
            std_y = y.std(0)*np.sqrt((n)/(n-1)) 
            ax.errorbar(x,mean_y,std_y,color=color,linestyle="none",marker="none",alpha=0.1)

        ax.xaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax=1))
        ax.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax=1))
        ax.set_title(to_map[wrt]+" in "+instance)
        ax.grid(True)
        ax.set_facecolor("whitesmoke")
fig.supxlabel("Quality")
fig.supylabel("Similarity")
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='outside lower right',ncols=2)
plt.show()

If the problem is globally convex, the expectation is that locally optimal solutions are very similar to eachother. However the similarity is low overall, which can stem from how the nature hamming distance and the size of the instances. Simply speaking, high values are not easy to achieve, even on the smaller instances.

# Conclusions

Overall it was quite suprising to see that the greedy version of local search performed better than the steepest, as I have not seen that often in practice. Althought the final quality was similar, the greedy version was much more efficient. In cases of instances like $\texttt{Esc32g}$ the heuristic was able to find the optimum along with local searches, as well as competing solutions for instances like $\texttt{Tai256c}$. 

No significant correlation between the quality of initial solution and final solution in local searches was found. No significant relation between the similarity and the final quality was found, but this could be partly due to how similarity is calculated.

Lastly there is value to be gained from running local search multiple times from different starting locations. After around 100 repetitions, which depends on the instance, the algorithm improves after which it stagnates.

# Difficulties

Coming up with an appropriate construction heuristic is not easy as it depends on the problem. In this case its not entirely intuitive on how to do it. The final solution is one that could probably be implemented for most problems, but does not imply good performance. In comparison a heuristic for TSP would add the closest city each time, which in a way is very similar to this solution!

There were some difficulties in how to properly pass the specific arguments to the solvers in the program. Finally it was achieved by implementing a ```Experiment class``` which first parses all command lines arguments and then using a ```switch```, launches the appropriate solver with its arguments.

# Introduced improvements

Initially the heuristic evaluated the whole incomplete solution. The improvement was to just evaluate assignment of the last 'location'. This was it was much faster, as one would expect for a construction heuristic, while not changing the final result. 